In [1]:
import os
import sys
import logging
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Any, List, Generator

In [2]:
# package imports
from vsim import electorate, candidates, simulation, voting_system

# Measuring fairness - Metric convergence

In [39]:
# setup helper for generating experiments (basically all sets of params to run through sim)
def generate_experiment_matrix(experiments: Dict[str, List[Any]]):
    return (dict(zip(experiments, x)) for x in itertools.product(*experiments.values()))

columns=[
    "electorate_size",
    "clusters",
    "cluster_std" 
    "issues",
    "candidates",
    "electoral_system",
    "apathy_prob",
    "fairness"
]

experiments = {
    "electorate_size": [10_000],
    "clusters": [10],
    "cluster_std": [1],
    "electoral_system": ["plurality", "majority"],
    "apathy_prob": [0.0],
    "candidates": [2],
    "issues": [2],
}

records = []
columns = [*experiments.keys(), "fairness", "t"]
experiment_matrix = list(generate_experiment_matrix(experiments))


# outer monte carlo sim
for _ in tqdm(range(10_000)):
    for params in experiment_matrix:

        # setup objects
        voters = electorate.setup_electorate(**params)
        parties = candidates.setup_candidates(**params, electorate=voters)
        system = voting_system.setup_voting_system(name=params["electoral_system"], **params)

        # simulate elections
        sim = simulation.VotingSimulator(electorate=voters, candidates=parties, system=system, log=logging.getLogger())

        result = sim.run()
        result_record = { "fairness": round(result.measured_fairness, 3), **params }
        result_record = {k: v for k, v in result_record.items() if k in columns}
        records.append(result_record)

100%|██████████████████████████████████████████| 10000/10000 [14:43<00:00, 11.31it/s]


In [32]:
results_df = pd.DataFrame.from_records(records)
results_df.head()

,fairness,electorate_size,clusters,cluster_std,electoral_system,apathy_prob,candidates,issues
0,0.920,10000,10,1,plurality,0.0,2,2
1,0.940,10000,10,1,plurality,0.0,2,2
2,0.933,10000,10,1,plurality,0.0,2,2
3,0.918,10000,10,1,plurality,0.0,2,2
4,0.851,10000,10,1,plurality,0.0,2,2


In [ ]:
# stoe away so we don't need to rerun experiment
filepath = "data/fairness-convergence-a91ed7aa-b6f6-4a85-9ccf-eeb37ea9da02.csv"
results_df.to_csv(filepath, index=False)

In [ ]:
# calculate cumulative averages for each electoral system
majority = results_df.electoral_system == "majority"
plurality = results_df.electoral_system == "plurality"

majority_df = results_df[majority].reset_index().drop(["index", axis=1])
plurality_df = results_df[plurality].reset_index().drop(["index", axis=1])

results_df.loc[, "fairness_cumsum"] = results_df[1:, "fairness_cumsum"] / results_df[1: ].index
results_df.loc[, "fairness_cumsum"] = results_df[1:, "fairness_cumsum"] / results_df[1: ].index